# **Youtube Videos Transcription with Faster Whisper**


[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/lewangdev/whisper-youtube/blob/main/faster_whisper_youtube.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/lewangdev/faster_whisper_youtube)](https://github.com/lewangdev/faster_whisper_youtube)


[faster-whisper](https://github.com/guillaumekln/faster-whisper) is a reimplementation of OpenAI's Whisper model using CTranslate2, which is a fast inference engine for Transformer models.

This implementation is up to 4 times faster than openai/whisper for the same accuracy while using less memory. The efficiency can be further improved with 8-bit quantization on both CPU and GPU.

Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

This Notebook will guide you through the transcription of a Youtube video using Faster Whisper. You'll be able to explore most inference parameters or use the Notebook as-is to store the transcript and video audio in your Google Drive.

In [1]:
#@markdown # **Check GPU type** 🕵️

#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime &rarr; Change runtime type &rarr; Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-349a03f6-7173-b773-b686-38d1a1144073)
Wed Nov 22 02:25:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [2]:
#@markdown # **Install libraries** 🏗️
#@markdown This cell will take a little while to download several libraries, including Faster Whisper.

#@markdown ---

! pip install faster-whisper
! pip install yt-dlp

import sys
import warnings
from faster_whisper import WhisperModel
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-

Using device: cuda:0


In [10]:
#@markdown # **Optional:** Save data in Google Drive 💾
#@markdown Enter a Google Drive path and run this cell if you want to store the results inside Google Drive.

# Uncomment to copy generated images to drive, faster than downloading directly from colab in my experience.
from google.colab import drive
drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"
#@markdown ---
drive_path = "\u7F85\u6D5A/done_txt" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change your Google Drive path.**

drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#@markdown # **Model selection** 🧠

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~0.8 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1.0 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~1.4 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~2.7 GB     |      ~2x       |
#@markdown | large-v1  |   1550 M   |        N/A         |      `large-v1`       |    ~4.3 GB     |       1x       |
#@markdown | large-v2  |   1550 M   |        N/A         |      `large-v2`       |    ~4.3 GB     |       1x       |

#@markdown ---
model_size = 'large-v2' #@param ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'medium.en', 'large-v1', 'large-v2']
device_type = "cuda" #@param {type:"string"} ['cuda', 'cpu']
compute_type = "int8_float16" #@param {type:"string"} ['float16', 'int8_float16', 'int8']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

model = WhisperModel(model_size, device=device_type, compute_type=compute_type)


In [11]:
#@markdown # **Video selection** 📺

#@markdown Enter the URL of the Youtube video you want to transcribe, wether you want to save the audio file in your Google Drive, and run the cell.

Type = "Google Drive" #@param ['Youtube video or playlist', 'Google Drive']
#@markdown ---
#@markdown #### **Youtube video or playlist**
URL = "" #@param {type:"string"}
# store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
video_path = "\u7F85\u6D5A/\u5F85\u8F49\u9010\u5B57\u7A3F" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

video_path_local_list = []

if Type == "Youtube video or playlist":

    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([URL])
        list_video_info = [ydl.extract_info(URL, download=False)]

    for video_info in list_video_info:
        video_path_local_list.append(Path(f"{video_info['id']}.wav"))

elif Type == "Google Drive":
    # video_path_drive = drive_mount_path / Path(video_path.lstrip("/"))
    video_path = drive_mount_path / Path(video_path.lstrip("/"))
    if video_path.is_dir():
        for video_path_drive in video_path.glob("**/*"):
            if video_path_drive.is_file():
                display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
            elif video_path_drive.is_dir():
                display(Markdown(f"**Subfolders not supported.**"))
            else:
                display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
            video_path_local = Path(".").resolve() / (video_path_drive.name)
            shutil.copy(video_path_drive, video_path_local)
            video_path_local_list.append(video_path_local)
    elif video_path.is_file():
        video_path_local = Path(".").resolve() / (video_path.name)
        shutil.copy(video_path, video_path_local)
        video_path_local_list.append(video_path_local)
        display(Markdown(f"**{str(video_path)} selected for transcription.**"))
    else:
        display(Markdown(f"**{str(video_path)} does not exist.**"))

else:
    raise(TypeError("Please select supported input type."))

for video_path_local in video_path_local_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


**/content/drive/My Drive/羅浚/待轉逐字稿/2892第一金_2022Q4法人說明會(英)_20230224.mp3 selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/2892第一金_2022Q3法人說明會(英)_20221130.mp3 selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/2892第一金_2022Q2法人說明會(英)_20220831.mp3 selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/2892第一金_2022Q1法人說明會(英)_20220531.mp3 selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/2892第一金_2021Q2法人說明會(英)_20210831.mp3 selected for transcription.**

...

**/content/drive/My Drive/羅浚/待轉逐字稿/4904遠傳_2013Q1法人說明會(英)_20130426.wma selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/4904遠傳_2012Q4法人說明會(英)_20130207.wma selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/4904遠傳_2012Q3法人說明會(英)_20121108.wma selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/4904遠傳_2012Q2法人說明會(英)_20120725.wma selected for transcription.**

**/content/drive/My Drive/羅浚/待轉逐字稿/4904遠傳_2012Q1法人說明會(英)_20120420.wma selected for transcription.**

In [ ]:
def seconds_to_time_format(s):
    # Convert seconds to hours, minutes, seconds, and milliseconds
    hours = s // 3600
    s %= 3600
    minutes = s // 60
    s %= 60
    seconds = s // 1
    milliseconds = round((s % 1) * 1000)

    # Return the formatted string
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"

#@markdown # **Run the model** 🚀

#@markdown Run this cell to execute the transcription of the video. This can take a while and very based on the length of the video and the number of parameters of the model selected above.

#@markdown ## **Parameters** ⚙️

#@markdown ### **Behavior control**
#@markdown #### Language
language = "auto" #@param ["auto", "en", "zh", "ja", "fr", "de"] {allow-input: true}
#@markdown #### initial prompt
initial_prompt = "Please do not translate, only transcription be allowed.  Here are some English words you may need: Cindy. And Chinese words: \u7206\u7834" #@param {type:"string"}
#@markdown ---
#@markdown #### Word-level timestamps
word_level_timestamps = False #@param {type:"boolean"}
#@markdown ---
#@markdown #### VAD filter
vad_filter = True #@param {type:"boolean"}
vad_filter_min_silence_duration_ms = 50 #@param {type:"integer"}
#@markdown ---
#@markdown #### Output(Default is srt, txt if `text_only` be checked )
text_only = True #@param {type:"boolean"}


# 遍歷 video_path_local_list，為每個文件進行轉錄
for video_path_local in video_path_local_list:
    # 轉換 MP4 文件到 WAV 格式（如果需要）
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])

    # 執行轉錄
    segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                      language=None if language == "auto" else language,
                                      initial_prompt=initial_prompt,
                                      word_timestamps=word_level_timestamps,
                                      vad_filter=vad_filter,
                                      vad_parameters=dict(min_silence_duration_ms=vad_filter_min_silence_duration_ms))

    display(Markdown(f"Detected language '{info.language}' with probability {info.language_probability}"))

    # 生成逐字稿文件
    ext_name = '.txt' if text_only else ".srt"
    transcript_file_name = video_path_local.stem + ext_name
    sentence_idx = 1
    with open(transcript_file_name, 'w') as f:
        for segment in segments:
            if word_level_timestamps:
                for word in segment.words:
                    ts_start = seconds_to_time_format(word.start)
                    ts_end = seconds_to_time_format(word.end)
                    print(f"[{ts_start} --> {ts_end}] {word.word}")
                    if not text_only:
                        f.write(f"{sentence_idx}\n")
                        f.write(f"{ts_start} --> {ts_end}\n")
                        f.write(f"{word.word}\n\n")
                    else:
                        f.write(f"{word.word} ")
                    sentence_idx += 1
            else:
                ts_start = seconds_to_time_format(segment.start)
                ts_end = seconds_to_time_format(segment.end)
                print(f"[{ts_start} --> {ts_end}] {segment.text}")
                if not text_only:
                    f.write(f"{sentence_idx}\n")
                    f.write(f"{ts_start} --> {ts_end}\n")
                    f.write(f"{segment.text.strip()}\n\n")
                else:
                    f.write(f"{segment.text.strip()}\n")
                sentence_idx += 1
        if text_only:
            f.write("\n")  # 確保在文本模式下換行


    try:
        shutil.copy(video_path_local.parent / transcript_file_name,
                    drive_whisper_path / transcript_file_name
        )
        display(Markdown(f"**Transcript file created: {drive_whisper_path / transcript_file_name}**"))
    except:
        display(Markdown(f"**Transcript file created: {video_path_local.parent / transcript_file_name}**"))


Detected language 'en' with probability 0.98779296875

[00:00:00,740 --> 00:00:07,940]  Hi, I'm Casey. Good afternoon, everyone. Welcome to join us for First Financial Holding 2022
[00:00:07,940 --> 00:00:14,820]  Four-Year Earnings Results. As usual, we will start with our presentation,
[00:00:15,870 --> 00:00:21,630]  including 2022 performance summary, financial highlights, and operating results.
[00:00:22,190 --> 00:00:29,150]  After the presentation, we'll invite Ms. Annie Lee, our IR head and head of IR,
[00:00:29,150 --> 00:00:35,390]  to proceed the Q&A session. You can raise your questions by typing at the bottom of the webcast
[00:00:35,390 --> 00:00:42,910]  window. Either in English or Chinese is fine with us. Or you can email us your questions
[00:00:42,910 --> 00:00:50,770]  after today's conference. Now I'd like to turn over to Mr. Keith Ko to begin today's presentation.
[00:00:50,770 --> 00:00:55,250]  Keith. Okay, thank you, Casey. Okay,
[00:00:55,250 --> 00:01:03,170]  please turn to slide five. This graph shows the net i

**Transcript file created: /content/drive/My Drive/羅浚/done_txt/2892第一金_2022Q4法人說明會(英)_20230224.txt**

Detected language 'en' with probability 0.9677734375

[00:00:00,820 --> 00:00:04,820]  Thank you for waiting. The meeting will begin shortly.
[00:00:07,630 --> 00:00:15,260]  Good afternoon, everyone. I'm Casey. Welcome to join us for First Financial Holding third quarter
[00:00:15,260 --> 00:00:23,580]  2022 webcast investor conference. Before we proceed the presentation, I'd like to disclose
[00:00:23,580 --> 00:00:31,380]  the following information. Starting from December 2015, in order to improve corporate governance
[00:00:32,100 --> 00:00:37,940]  First Financial Holding has set out ethical corporate management best practice principles
[00:00:37,940 --> 00:00:44,660]  and conducting procedures and guidelines. For more information, please refer to our website
[00:00:44,660 --> 00:00:53,570]  www.ffhec.com.tw. Okay, let's start with our performance presentation.
[00:00:55,490 --> 00:01:02,210]  Presentation material can be downloaded from our website and one year replay will be available
[00:01:02,210 --> 00:01:11,100]  after today's 

**Transcript file created: /content/drive/My Drive/羅浚/done_txt/2892第一金_2022Q3法人說明會(英)_20221130.txt**